# Imported  required libraries

In [1]:
from keras.preprocessing import image as image_utils
import argparse
import cv2
import numpy as np
import os
import random
import sys
from queue import Queue
import threading

from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

from keras.layers import *
from keras.models import Model

from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
import h5py

# TRAIN THE MODEL:

In [2]:
def train_model():
    model_resnet50_conv = ResNet50(weights='imagenet', include_top=False)
 

    #Create your own input format (here 3x224x224)
    input = Input(shape=(224,224,3),name = 'image_input')

    #Use the generated model 
    output_resnet50_conv = model_resnet50_conv(input)

    #Add the fully-connected layers 
    x = Flatten()(output_resnet50_conv)
    #x = Dropout(0.3)(x)
    x = Dense(10, activation='softmax', name='predictions')(x)

    #Create your own model 
    model = Model(input,x)

    #In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
   
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [3]:
def preprocess_input(x, dim_ordering='default'):
    
    x=np.array(x,dtype=np.uint8)
        
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    return x




In [4]:
# decode_predictions() function takes a numpy array of predictions as input and returns the class labels corresponding to the highest prediction probabilities.
def decode_predictions(preds):
    global CLASS_INDEX
    assert len(preds.shape) == 2 and preds.shape[1] == 10
    
    if CLASS_INDEX is None:
        fpath = get_file('/Users/sonukumar/Desktop/Distracted_Driver_Detection-master/model.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    
    indices = np.argmax(preds)
    results = {'c0': 'normal driving','c1': 'texting - right','c2': 'talking on the phone - right','c3': 'texting - left','c4': 'talking on the phone - left','c5':'operating the radio','c6': 'drinking','c7': 'reaching behind','c8': 'hair and makeup','c9': 'talking to passenger'}  
    return results(indices)

In [5]:
import queue
label = '0'
# lock is an object provided by the threading module in Python that helps in managing access to shared resources in a multi-threaded environment
lock = threading.Lock()
# frame_queue and label_queue are used to hold incoming video frames and their corresponding predicted labels,
frame_queue = queue.Queue()
label_queue = queue.Queue(maxsize=1)


results = {'0': 'normal driving','1': 'texting - right','2': 'talking on the phone - right','3': 'texting - left','4': 'talking on the phone - left','5':'operating the radio','6': 'drinking','7': 'reaching behind','8': 'hair and makeup','9': 'talking to passenger'}  
kfold_weights_path = os.path.join('/Users/sonukumar/Desktop/Distracted_Driver_Detection-master/weights_1.h5')
class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        global label
        
        print("[INFO] loading network...")
        self.model = train_model()
        self.model.load_weights(kfold_weights_path)
        while True:
            try:
                frame = frame_queue.get(timeout=1)
            except queue.Empty:
                continue

            if frame is None:
                break
                
            label = self.predict(frame)
            
            with lock:
                label_queue.put(label)

    def predict(self, frame):
        X_test = []
        X_test.append(frame)
        test_data = np.array(X_test, dtype=np.uint8)
        test_data = test_data.astype('float16')   
        mean_pixel = [103.939, 116.779, 123.68]
        test_data[:, :, :, 0] -= mean_pixel[0]   
        test_data[:, :, :, 1] -= mean_pixel[1]
        test_data[:, :, :, 2] -= mean_pixel[2]
	
        preds = self.model.predict(test_data)
#         np.argmax function is used to find the index of the maximum value in the preds array, which corresponds to the predicted class. 
        result=str(np.argmax(preds))
        return result

cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture('/Users/sonukumar/Desktop/Distracted_Driver_Detection-master/recorded.mp4')
if (cap.isOpened()):
    print("Camera OK")
else:
    cap.open()

keras_thread = MyThread()
keras_thread.start()

while (True):
    ret, original = cap.read()

    if original is not None:
        frame = cv2.resize(original, (224, 224))
        frame_queue.put(frame)

        label = label_queue.get()
        cv2.putText(original, "Label: {}".format(label), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(original,"Distraction: {}".format(results[label]), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Classification", original)

    if (cv2.waitKey(4) & 0xFF == ord('q')):
        break;

frame_queue.put(None)
keras_thread.join()

cap.release()
cv2.destroyAllWindows()
sys.exit()

Camera OK
[INFO] loading network...


/Users/sonukumar/opt/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 213ms/step


1/1 [==============================] - 0s 343ms/step


1/1 [==============================] - 0s 387ms/step


...

1/1 [==============================] - 0s 217ms/step


1/1 [==============================] - 0s 222ms/step


1/1 [==============================] - 0s 234ms/step


SystemExit: 

/Users/sonukumar/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
